In [1]:
import os
import re

import numpy as np
import pandas as pd

# os.chdir("/home/yujia/Project/2023-07-20-individual_MR/dat/02_base_data/")

In [2]:
TC = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/download/without_UKB_TC_INV_SAS_1KGP3_ALL.meta.singlevar.results.gz", sep="\t", compression = 'gzip')

print("Total SNP in the GWAS data: ")
print(TC.shape)

Total SNP in the GWAS data: 
(15402860, 14)


In [3]:
TC

,rsID,CHROM,POS_b37,REF,ALT,N,N_studies,POOLED_ALT_AF,EFFECT_SIZE,SE,pvalue_neg_log10,pvalue,pvalue_neg_log10_GC,pvalue_GC
0,rs189800799,1,701835,T,C,2229,2,0.03360,0.075323,0.094135,0.373022,0.424,0.361598,0.435
1,rs373285923,1,712547,G,C,2229,2,0.03410,0.103932,0.115536,0.433736,0.368,0.420169,0.380
2,rs12071806,1,712762,T,G,2229,2,0.03880,0.112107,0.088854,0.683909,0.207,0.661050,0.218
3,NaN,1,713044,C,<CN2>,2229,2,0.03340,0.078736,0.094692,0.391800,0.406,0.379719,0.417
4,rs200310424,1,713131,A,AT,2229,2,0.03360,0.073630,0.090531,0.380867,0.416,0.369169,0.427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15402855,rs45475396,22,51222766,G,A,13216,5,0.01890,-0.006544,0.044170,0.054422,0.882,0.053033,0.885
15402856,rs373624689,22,51228439,G,A,8882,3,0.04490,-0.025910,0.041218,0.276047,0.530,0.267926,0.540
15402857,rs144549712,22,51229855,G,A,8882,3,0.13200,0.011026,0.029001,0.152547,0.704,0.148351,0.711
15402858,rs200189535,22,51235959,T,C,8882,3,0.27200,0.003486,0.025826,0.049322,0.893,0.048069,0.895


In [4]:
# rename the data (REF: non-effect allele; ALT: effect allele)
TC.rename(columns={"CHROM":"CHR", "POS_b37":"BP", "rsID":"SNP", "REF":"A2", "ALT":"A1", "POOLED_ALT_AF":"EAF", "EFFECT_SIZE":"BETA", "SE":"SE", "pvalue":"P", "N":"N"}, inplace=True)

# convert EAF to MAF
TC['MAF'] = TC['EAF'].apply(lambda x : x if x <= 0.5 else 1-x)

# Uppercase the A1 and A2
TC['A1'] = TC['A1'].str.upper()
TC['A2'] = TC['A2'].str.upper()

In [5]:
TC

,SNP,CHR,BP,A2,A1,N,N_studies,EAF,BETA,SE,pvalue_neg_log10,P,pvalue_neg_log10_GC,pvalue_GC,MAF
0,rs189800799,1,701835,T,C,2229,2,0.03360,0.075323,0.094135,0.373022,0.424,0.361598,0.435,0.03360
1,rs373285923,1,712547,G,C,2229,2,0.03410,0.103932,0.115536,0.433736,0.368,0.420169,0.380,0.03410
2,rs12071806,1,712762,T,G,2229,2,0.03880,0.112107,0.088854,0.683909,0.207,0.661050,0.218,0.03880
3,NaN,1,713044,C,<CN2>,2229,2,0.03340,0.078736,0.094692,0.391800,0.406,0.379719,0.417,0.03340
4,rs200310424,1,713131,A,AT,2229,2,0.03360,0.073630,0.090531,0.380867,0.416,0.369169,0.427,0.03360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15402855,rs45475396,22,51222766,G,A,13216,5,0.01890,-0.006544,0.044170,0.054422,0.882,0.053033,0.885,0.01890
15402856,rs373624689,22,51228439,G,A,8882,3,0.04490,-0.025910,0.041218,0.276047,0.530,0.267926,0.540,0.04490
15402857,rs144549712,22,51229855,G,A,8882,3,0.13200,0.011026,0.029001,0.152547,0.704,0.148351,0.711,0.13200
15402858,rs200189535,22,51235959,T,C,8882,3,0.27200,0.003486,0.025826,0.049322,0.893,0.048069,0.895,0.27200


In [6]:
# reorder the column
cols = ["CHR", "BP", "SNP", "A1", "A2", "N", "SE", "P", "BETA", "MAF", "EAF"]
TC = TC[cols]

# Remove NaN row
TC = TC.dropna()

# convert to correct column type
TC = TC.astype({'CHR':'int', 'BP':'int', "SNP":'str', 'A1':'str', 'A2':'str', 'N':'int', 'SE':'float', 'P':'float', 'BETA':'float', 'MAF':'float', 'EAF':'float'})

# remove SNP without SNPID
TC = TC[TC['SNP'].str.startswith('rs')]

# save the formatted results
TC.to_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC_formated_nadrop.tsv.gz", sep="\t", index=False)

print("SNP left after basic QC: ")
print(TC.shape)

SNP left after basic QC: 
(15316327, 11)


In [7]:
TC

,CHR,BP,SNP,A1,A2,N,SE,P,BETA,MAF,EAF
0,1,701835,rs189800799,C,T,2229,0.094135,0.424,0.075323,0.03360,0.03360
1,1,712547,rs373285923,C,G,2229,0.115536,0.368,0.103932,0.03410,0.03410
2,1,712762,rs12071806,G,T,2229,0.088854,0.207,0.112107,0.03880,0.03880
4,1,713131,rs200310424,AT,A,2229,0.090531,0.416,0.073630,0.03360,0.03360
5,1,714019,rs114983708,G,A,2229,0.085180,0.205,0.107875,0.03880,0.03880
...,...,...,...,...,...,...,...,...,...,...,...
15402855,22,51222766,rs45475396,A,G,13216,0.044170,0.882,-0.006544,0.01890,0.01890
15402856,22,51228439,rs373624689,A,G,8882,0.041218,0.530,-0.025910,0.04490,0.04490
15402857,22,51229855,rs144549712,A,G,8882,0.029001,0.704,0.011026,0.13200,0.13200
15402858,22,51235959,rs200189535,C,T,8882,0.025826,0.893,0.003486,0.27200,0.27200


In [8]:
# =============================
# Step 2: remove SNPs with MAF < 0.01
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC_formated_nadrop.tsv.gz |" + \
      "awk 'NR==1 || ($10 > 0.01) {print}' |" + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.standardGWASQC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.standardGWASQC.gz | wc -l"
print("SNP left after removing SNPs with MAF < 0.01: ")
print(os.popen(cmd).read())

SNP left after removing SNPs with MAF < 0.01: 
9274486



In [9]:
# =============================
# Step 3: remove duplicate SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.standardGWASQC.gz |" + \
      "awk '{seen[$3]++; if(seen[$3]==1){ print}}' |" + \
      "gzip - > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.nodup.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.nodup.gz | wc -l"
print("SNP left after removing duplicate SNPs: ")
print(os.popen(cmd).read())

SNP left after removing duplicate SNPs: 
9254285



In [10]:
# =============================
# Step 4: keep non-ambiguous SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.nodup.gz | " + \
      "awk '!( ($4==\"A\" && $5==\"T\") || ($4==\"T\" && $5==\"A\") || ($4==\"G\" && $5==\"C\") || ($4==\"C\" && $5==\"G\")) {print}' | " + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.QC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_SAS/TC.QC.gz | wc -l"
print("SNP left after removing ambiguous SNPs: ")
print(os.popen(cmd).read())

SNP left after removing ambiguous SNPs: 
7964843

